In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

Using TensorFlow backend.


In [2]:
batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [4]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

In [5]:
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [6]:
# RMSProp optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Train model using RMSProp
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [7]:
import numpy as np

In [8]:
#convert maxtrix into an m * n matrix
adj_x_train = np.zeros((3072,x_train.shape[0]))
for g in range(x_train.shape[0]):
    l = x_train[g]
    for h in range(x_train.shape[3]):
        for j in range(x_train.shape[1]):
            for i in range(x_train.shape[2]):
                adj_x_train[((h*1024) + (j*32) + (i)),g] = l[j, i, h]
print('finished')

finished


In [9]:
adj_x_test = np.zeros((3072,x_test.shape[0]))
for g in range(x_test.shape[0]):
    l = x_test[g]
    for h in range(x_test.shape[3]):
        for j in range(x_test.shape[1]):
            for i in range(x_test.shape[2]):
                adj_x_test[((h*1024) + (j*32) + (i)),g] = l[j, i, h]
print('finished')

finished


In [10]:
x_test_new = adj_x_test.T
print('finished')

finished


In [11]:
x_train_new = adj_x_train.T
print('finished')

finished


In [12]:
from sklearn.decomposition import PCA
pca_100 = PCA(n_components = 15)
pca_result_100 = pca_100.fit_transform(x_train_new)
X_train_pca = pca_100.transform(x_train_new)
X_recon_train = X_train_pca.dot(pca_100.components_) + pca_100.mean_
print('finished-2')


finished-2


In [13]:
# perform pca on our m x n matrices
pca_100_test = PCA(n_components = 15)
pca_result_100_test = pca_100_test.fit_transform(x_test_new)
X_test_pca = pca_100_test.transform(x_test_new)
X_recon_test = X_test_pca.dot(pca_100_test.components_) + pca_100_test.mean_
print('finished')

finished


In [14]:
def con_4d(X):
    X_4D = np.zeros((X.shape[0],32,32,3))
    for m in range(X.shape[0]):
        for n in range(X.shape[1]):
            #print(m, int(n/32), n % 32, int(n/1024))
            X_4D[m, (int(n/32) - int(n/1024) * 32),(n % 32), int(n/1024)] = X[m,n]
    return X_4D

In [15]:
# convert the testing and training data back into 4D array
x_train = con_4d(X_recon_train)
x_test = con_4d(X_recon_test)
print('finished')


finished


In [16]:
## Variations of data augmentation ##

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # Real time augmentation
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

Using real-time data augmentation.


In [17]:
 model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),workers=4,steps_per_epoch=len(x_test)/32)


# Scores for model
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Epoch 1/100
313/312 [==============================] - 13s 40ms/step - loss: 2.1754 - acc: 0.1762 - val_loss: 2.0587 - val_acc: 0.2345
Epoch 2/100
313/312 [==============================] - 9s 28ms/step - loss: 2.0474 - acc: 0.2349 - val_loss: 1.9869 - val_acc: 0.2921
Epoch 3/100
313/312 [==============================] - 8s 27ms/step - loss: 1.9866 - acc: 0.2751 - val_loss: 1.9532 - val_acc: 0.2932
Epoch 4/100
313/312 [==============================] - 8s 27ms/step - loss: 1.9526 - acc: 0.2900 - val_loss: 1.8865 - val_acc: 0.3251
Epoch 5/100
313/312 [==============================] - 8s 27ms/step - loss: 1.9284 - acc: 0.2981 - val_loss: 1.8548 - val_acc: 0.3334
Epoch 6/100
313/312 [==============================] - 8s 27ms/step - loss: 1.9027 - acc: 0.3077 - val_loss: 1.8324 - val_acc: 0.3471
Epoch 7/100
313/312 [==============================] - 8s 27ms/step - loss: 1.8688 - acc: 0.3139 - val_loss: 1.7938 - val_acc: 0.3574
Epoch 8/100
313/312 [==============================] - 9s 27m

Epoch 70/100
313/312 [==============================] - 9s 29ms/step - loss: 1.6825 - acc: 0.3907 - val_loss: 1.6090 - val_acc: 0.4286
Epoch 71/100
313/312 [==============================] - 9s 28ms/step - loss: 1.6646 - acc: 0.4019 - val_loss: 1.6022 - val_acc: 0.4285
Epoch 72/100
313/312 [==============================] - 9s 29ms/step - loss: 1.6610 - acc: 0.4012 - val_loss: 1.6536 - val_acc: 0.4167
Epoch 73/100
313/312 [==============================] - 9s 29ms/step - loss: 1.6705 - acc: 0.3981 - val_loss: 1.6036 - val_acc: 0.4283
Epoch 74/100
313/312 [==============================] - 9s 29ms/step - loss: 1.6586 - acc: 0.4071 - val_loss: 1.5953 - val_acc: 0.4288
Epoch 75/100
313/312 [==============================] - 9s 29ms/step - loss: 1.6734 - acc: 0.3974 - val_loss: 1.6106 - val_acc: 0.4235
Epoch 76/100
313/312 [==============================] - 9s 29ms/step - loss: 1.6738 - acc: 0.3950 - val_loss: 1.6114 - val_acc: 0.4289
Epoch 77/100
313/312 [==============================] -

In [ ]:
print(scores[1])